# IndicTrans2 HF Inference

We provide an example notebook on how to use our IndicTrans2 models which were originally trained with the fairseq to HuggingFace transformers for inference purpose.


## Setup

Please run the cells below to install the necessary dependencies.


In [ ]:
%%capture
!git clone https://github.com/AI4Bharat/IndicTrans2.git

In [ ]:
%%capture
%cd /content/IndicTrans2/huggingface_interface

In [ ]:
%%capture
!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer
!python3 -c "import nltk; nltk.download('punkt')"
!python3 -m pip install bitsandbytes scipy accelerate datasets
!python3 -m pip install sentencepiece

!git clone https://github.com/VarunGumma/IndicTransTokenizer
%cd IndicTransTokenizer
!python3 -m pip install --editable ./
%cd ..

**IMPORTANT : Restart your run-time first and then run the cells below.**

## Inference


In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer

BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None

In [ ]:
def initialize_model_and_tokenizer(ckpt_dir, direction, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = IndicTransTokenizer(direction=direction)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            src=True,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text
        generated_tokens = tokenizer.batch_decode(generated_tokens.detach().cpu().tolist(), src=False)

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

### English to Indic Example


In [ ]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, "en-indic", quantization)

ip = IndicProcessor(inference=True)

en_sents = [
    # trying out different locations but always gives जाता था in male tone.
    "When I was young, I used to go to the park every day.",
    "When I was young, I used to go to the mall every day.",
    "When I was young, I used to go to the school every day.",
    "When I was young, I used to go to the garden every day.",
    "When I was young, I used to go to the library every day.",
    "When I was young, I used to go to the parlour every day.",
    "When I was young, I used to go to the salon every day.",
    "When I was young, I used to go to the hospital every day.",
    # trying spl symbols. its ignoring them. DOESN'T convert Numbers to Hindi
    "*** let's meet at 1:30pm today.",
    "*** let's meet at 1:30pm today. ***",
    #famous tag line "Paytm Karo" could not detect?? - detected but politely
    "You owe me ₹1234. Paytm me.",
    "You owe me ₹1234. Do Paytm me.",
    "Good morning",

    #sound rude but LLM can't
    "Return my money! Else you are doomed!!!",
# we need to convert "Indian" english and not actual American native english to Inidc lang



#can do joint sentences with diff context
    "He has many old books, which he inherited from his ancestors. I can't figure out how to solve my problem.",
    "She is very hardworking and intelligent, which is why she got all the good marks.",
    "She is very hardworking and inteLLigent, which is y she got all the g00d marks.",

    "She is very hardworking and inteLLigent, which is Y she got all the g00d marks.",
    "We watched a new movie last week, which was very inspiring.",
    "If you had met me at that time, we would have gone out to eat.",
    "She went to the market with her sister to buy a new sari.",
    "Raj told me that he is going to his grandmother's house next month.",
    "All the kids were having fun at the party and were eating lots of sweets.",
    "My friend has invited me to his birthday party, and I will give him a gift.",
# for healthcare applications
    "My back is hurting real bad",
    "The pus oozes out from the swollen thing that I have on my left thumb",
    "I can't breathe properly when I walk fast",
    "I am not feeling good, I just don't feel like going out of my room",
    "I think I am pregnant. Can you please check?",
    "I think he is pregnant",
    "I am suffering from a severe hairloss. Dr. please suggest a solution for me",
    "I want my xray to be done for my leg",
    "My pinky finger hurts badly", #
    "Ouch my calf hurts!!!!!", #
    "My cow gave birth to a little calf yesterday and we named him moooo",
    "My heels really hurts",
    "The wound on my ankle is still fresh",
    "Intestine, Pancreas, bladder, spleen, hips, wrist , elbow, palm, upper arm, forearm, shoulder",
    "Adam's Apple" ,#
    "Abdomen , chest, spine, stomach", # both abdomen and stomach = पेट
    "Glands such as the pituitary, thyroid, and adrenal glands",
    #Identified sole is being used as a noun or adjective
    "My sole is aching real bad",
    "The sole reason I left my job is to go for higher studies" ,
    "I missed my flight",
    "Miss Sofia won the contest",
    "I miss you Sofia",
    "I feel truly sorry for you",
    "It's raining cats and dogs",#
    "His wardrobe was at sixes and sevens",#
    "My wife won't let me speak to my mother",#
    "My husband won't let me speak to my mother",#
    "My husband took me to Musoorie for holidays",
    "My wife took me to Thailand for holidays",
    "My wife gifted me a INR 100 thousand purse",
    "She gifted me a INR 100 thousand purse",
    "He gifted me a INR 100 thousand purse",
    "I have got to buy a computer with larger RAM",
    "Ram of my new computer is slightly bigger than my old computer",
    "My Algorithm is better than yours",
    "Algorithm: A step-by-step set of instructions or rules followed to perform a task or solve a problem.",
    "Big Data: Extremely large and complex data sets that cannot be easily managed or processed with traditional data processing tools.",
    "Machine Learning: A subset of artificial intelligence that focuses on the development of algorithms and statistical models that enable computers to improve their performance on a task through experience.",
    "Data Mining: The process of discovering patterns and knowledge from large amounts of data.",
    "Data Science: The interdisciplinary field that uses scientific methods, processes, algorithms, and systems to extract insights and knowledge from structured and unstructured data.",
    "Data Analytics: The process of examining, cleaning, transforming, and modeling data to extract useful information, draw conclusions, and support decision-making.",
    "Artificial Intelligence (AI): The simulation of human intelligence in machines that are programmed to think and learn like humans.",
    "High-Performance Computing (HPC): The use of supercomputers and parallel processing techniques to solve complex computational problems.",
    "Cloud Computing: The delivery of computing services, including storage, processing power, and analytics, over the internet.",
    "Distributed Computing: A field of computer science that deals with designing and implementing systems that involve multiple interconnected computers working together.",
    "Parallel Computing: The simultaneous execution of multiple calculations or processes to solve a complex problem more efficiently.",
    "Deep Learning: A subset of machine learning that involves neural networks with multiple layers (deep neural networks) to learn and make decisions.",
    "Natural Language Processing (NLP): The field of artificial intelligence that focuses on the interaction between computers and humans using natural language.",
    "Predictive Modeling: The process of creating and testing a model to predict future outcomes based on historical data.",
    "Quantum Computing: A type of computing that uses the principles of quantum mechanics to perform calculations, offering potential advantages for certain types of problems.",
    "Computational Biology: The application of computational techniques to the understanding and modeling of biological systems.",
    "Network Analysis: The study of relationships and connections between nodes in a network, often represented as graphs.",
    "Data Visualization: The representation of data in graphical or visual formats to help people understand the patterns, trends, and insights within the data.",
    "Hadoop: An open-source framework for distributed storage and processing of large data sets.",
    "Cybersecurity: The practice of protecting computer systems, networks, and data from security breaches and unauthorized access.",
    "My brother-in-law helped me in cleaning the space for the guests",
    "My sister-in-law helped me in cleaning the space for the guests",
    "My grandmother cooks delicious meals to us",
    "My grandfather is a man of intelligence",
    "We went to our grandmother's house yesterday",
    "We will go to our aunt's today",
    "My niece is so cute that I want to squish her face everytime I see her",# neice = nees(in hindi)
    "My nephew started going to school last week",
    "My aunt goes to gym everyday",
    "My aunt cooks really delicious meals", #aunt -- chachi, uncle -- chacha
    "My sister-in-law told me that she is thinking of joining the gym", #sister-in-law -- saali, #brother-in-law-- jeeja
    "My uncle asked me to stay",
    "My maternal gradmother cooks delicious food for us",
    "My maternal uncle owns a shoe store",
    "My brother loves eating chocolate",
    "My brother's wife loves chocolate",
    "My cousins danced at my brother's wedding",
    "She sells seashells on the sea shore",
    "My sister in law is equivalent to my mother",#
    "My mother is my grandmother's daughter ",#
    "My washbasin is white in color",
    "My brother's mother's daughter's only brother's wife's son's mother is my sister-in-law",#
    "my brother's wife is my sister in law",#
    "I have always wanted to go to Saharanpur during winters",
    "Have you ever had Chhole bhature with Lassi in New Delhi?",
    "I live in Jafar Nawaz",
    "My father owns a shop named Naveen Stationers in Saharanpur",
    "Hey, did you watch the movie Kesariya?",
    "I can't believe you have never watched Kabhi khushi kabhi gum.",
    "Did you take a bath today? You are smelling awful today.",
    "You are a very reliable person in my life who I can tell all my secrets to."












]

src_lang, tgt_lang = "eng_Latn", "hin_Deva"
hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(en_sents, hi_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
del en_indic_tokenizer, en_indic_model


eng_Latn - hin_Deva
eng_Latn: When I was young, I used to go to the park every day.
hin_Deva: जब मैं छोटा था, मैं हर दिन पार्क जाता था।
eng_Latn: When I was young, I used to go to the mall every day.
hin_Deva: जब मैं छोटा था, मैं हर दिन मॉल जाता था।
eng_Latn: When I was young, I used to go to the school every day.
hin_Deva: जब मैं छोटा था, मैं हर दिन स्कूल जाता था।
eng_Latn: When I was young, I used to go to the garden every day.
hin_Deva: जब मैं छोटा था, मैं हर दिन बगीचे में जाता था।
eng_Latn: When I was young, I used to go to the library every day.
hin_Deva: जब मैं छोटा था, मैं हर दिन पुस्तकालय जाता था।
eng_Latn: When I was young, I used to go to the parlour every day.
hin_Deva: जब मैं छोटा था, तो मैं हर दिन पार्लर जाता था।
eng_Latn: When I was young, I used to go to the salon every day.
hin_Deva: जब मैं छोटा था, मैं हर दिन सैलून जाता था।
eng_Latn: When I was young, I used to go to the hospital every day.
hin_Deva: जब मैं छोटा था, तो मैं हर दिन अस्पताल जाता था।
eng_Latn: *** let's m

### Indic to English Example

In [ ]:
indic_en_ckpt_dir = "ai4bharat/indictrans2-indic-en-1B"  # ai4bharat/indictrans2-indic-en-dist-200M
indic_en_tokenizer, indic_en_model = initialize_model_and_tokenizer(indic_en_ckpt_dir, "indic-en", "")

ip = IndicProcessor(inference=True)

hi_sents = [
    'जब लाइफ़ हो आउट ऑफ कंट्रोल होठों को करके गोल होठों को करके गोल सीटी बजाके बोल आल इज़ वेल',
    'मेरे भाई की माँ की बेटी की इकलौती भाई की पत्नी के बेटे की माँ मेरी भाभी है',
    "जब मैं छोटा था, मैं हर रोज़ पार्क जाता था।",
    "उसके पास बहुत सारी पुरानी किताबें हैं, जिन्हें उसने अपने दादा-परदादा से विरासत में पाया।",
    "मुझे समझ में नहीं आ रहा कि मैं अपनी समस्या का समाधान कैसे ढूंढूं।",
    "वह बहुत मेहनती और समझदार है, इसलिए उसे सभी अच्छे मार्क्स मिले।",
    "हमने पिछले सप्ताह एक नई फिल्म देखी जो कि बहुत प्रेरणादायक थी।",
    "अगर तुम मुझे उस समय पास मिलते, तो हम बाहर खाना खाने चलते।",
    "वह अपनी दीदी के साथ बाजार गयी थी ताकि वह नई साड़ी खरीद सके।",
    "राज ने मुझसे कहा कि वह अगले महीने अपनी नानी के घर जा रहा है।",
    "सभी बच्चे पार्टी में मज़ा कर रहे थे और खूब सारी मिठाइयाँ खा रहे थे।",
    "मेरे मित्र ने मुझे उसके जन्मदिन की पार्टी में बुलाया है, और मैं उसे एक तोहफा दूंगा।",
    "आप पर मेरा ₹1234 बकाया है।  पेटीएम करें।",
    "वह समुद्र तट पर समुद्री गोले बेचती है।",
    "मेरे चचेरे भाइयों ने मेरे भाई की शादी में नृत्य किया",
    "मेरे भाभी को चॉकलेट पसंद है",
    "मेरे भाई को चॉकलेट खाना बहुत पसंद है।",
    "मेरे मामा की जूतों की दुकान है।",
    "मेरी नानी हमारे लिए स्वादिष्ट खाना बनाती हैं।",
    "मेरी ताई जी रोज जिम जाती हैं।",
    "मेरे जीजा ने मेहमानों के लिए जगह साफ करने में मेरी मदद की।",
    "मेरी बड़ी माँ ने मेहमानों के लिए जगह साफ करने में मेरी मदद की",
    "मेरी दादी हमारे लिए स्वादिष्ट भोजन बनाती हैं।",
    "मेरे दादा एक बुद्धिमान व्यक्ति हैं।",
    "हम कल अपनी दादी के घर गए थे।",
    "हम आज अपनी चाची के पास जाएँगे।",
    "मेरी नीस इतनी प्यारी है कि जब भी मैं उसे देखता हूं तो मैं उसका चेहरा निचोड़ना चाहता हूं।",
    "मेरे भतीजे ने पिछले हफ्ते स्कूल जाना शुरू कर दिया।"
    "यह बच्चा मेरी आँखों का तारा है",
    "उसकी क्या बात कर रहे हो, वह तो मेरे खून का प्यासा हो गया है।",
    "आतंकवादियों को देखकर मेरा तो खून ठण्डा पड़ गया।",
    "आई.पी.एस पास करके दीक्षा ने सचमुच गढ़ फतह कर लिया।",
    "कार्तिक गधे को बाप बनाना अच्छी तरह जानता हैं।",
    "अभी आपको मेरी बात समझ में नहीं आ रही, जब जूते पड़ेंगे तब समझ में आएगी।",
    "जहाँ चाह, वहाँ राह।",
    "हर कड़ी में राह होती है।",
    "काम का बोझ भारी होता है।",
    "रोम एक दिन में नहीं बना था।",
    "अब पछताए होत क्या जब चिड़ीया चुग गई खेत।",
    "हर चमकते चीज सोना नहीं होती।",
    "दो गलत सही नहीं बनाते।",
    "जहाँ धूम है, वहाँ आग होगी।",
    "विद्यार्थी स्कूल जाने का समय है। यह बहुत महत्वपूर्ण है कि आप समय पर स्कूल पहुंचें ताकि आप अपने पढ़ाई में निरंतरता बनाए रख सकें। स्कूल जाना आपके शिक्षा के लिए आवश्यक है और यह आपके भविष्य को सुनिश्चित करने में मदद कर सकता है। इसलिए, आप आज के दिन को सकारात्मकता के साथ शुरू करें और स्कूल के लिए समय पर निकलें। शुभ कामनाएं!",
    "शुभ प्रभात! आपका दिन सुखमय और सफल हो",
    "दिव्यांग व्यक्तियों की मदद करना सामाजिक जिम्मेदारी है।",
    "इन व्यक्तियों को समाज में समाहित करने और उन्हें समर्थ बनाने के लिए हमें सकारात्मक कदम उठाने चाहिए",
    "दिव्यांग व्यक्तियों के साथ सहानुभूति और समर्थन बनाए रखना अत्यंत महत्वपूर्ण है।",
    "उन्हें समाज में पूरी तरह से शामिल करना और उनकी सामाजिक समाहिति को बढ़ावा देना।",
    "उन्हें उच्च शिक्षा और प्रशिक्षण के अवसरों का भी लाभ दिलाना।",
    "सरकार और सामाजिक संगठनों से उनके लिए सुविधाएं और योजनाएं मांगना और उन्हें इनका लाभ उठाने में मदद करना।",
    "उन्हें रोजगार के अवसरों को सुझाना और उन्हें समर्थ बनाने के लिए तकनीकी और व्यावासायिक प्रशिक्षण प्रदान करना।",
    "सामाजिक संगठनों में सक्रियता दिखाना और दिव्यांगता के मुद्दों पर जागरूकता फैलाना।",
    "यह सभी कदम सामूहिक रूप से समृद्धि और समाज में समाहिति की दिशा में सहायक हो सकते हैं।",
    "कनक कनक ते सौ गुनी, मादकता अधिकाय। वा खाए बौराय जग, या पाए बौराय।",
    "उसकी मुस्कान सूर्य की किरणों की तरह चमक रही थी।",
    "समय एक अजीब सा दरवाज़ा है, जो कभी फिर नहीं खुलता।",
    "बादल बढ़ रहे हैं, बारिश बरस रही है।",
    "बस बुढ़िया बैठी बाल कांपा कर बोली।",
    "बाग़ के फूलों ने रात को महका दिया।",
    "गंगा जी की धारा, स्वर्ग से भी प्यारी है।",
    "सुरमा स्वर्ग से सजीवनी लेकर आई।",
    "उसकी आँखों में आसमान का रंग और उसके बोल में दरिया की गहराई थी",
    "हनुमान की पूंछ में लगन ना पाई आग, लंका सारी जल गई, गए निशाचर भग"

    ]
# hi_sents = hi_translations
src_lang, tgt_lang = "hin_Deva", "eng_Latn"
en_translations = batch_translate(hi_sents, src_lang, tgt_lang, indic_en_model, indic_en_tokenizer, ip)


print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(hi_sents, en_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
del indic_en_tokenizer, indic_en_model


hin_Deva - eng_Latn
hin_Deva: जब लाइफ़ हो आउट ऑफ कंट्रोल होठों को करके गोल होठों को करके गोल सीटी बजाके बोल आल इज़ वेल
eng_Latn: When life is out of control, make lips round, make lips round, make whistles round, say all is well.
hin_Deva: मेरे भाई की माँ की बेटी की इकलौती भाई की पत्नी के बेटे की माँ मेरी भाभी है
eng_Latn: My brother's mother's daughter's only brother's wife's son's mother is my sister-in-law
hin_Deva: जब मैं छोटा था, मैं हर रोज़ पार्क जाता था।
eng_Latn: When I was young, I used to go to the park every day.
hin_Deva: उसके पास बहुत सारी पुरानी किताबें हैं, जिन्हें उसने अपने दादा-परदादा से विरासत में पाया।
eng_Latn: She has a lot of old books, which she inherited from her grandparents.
hin_Deva: मुझे समझ में नहीं आ रहा कि मैं अपनी समस्या का समाधान कैसे ढूंढूं।
eng_Latn: I don't know how to find a solution to my problem.
hin_Deva: वह बहुत मेहनती और समझदार है, इसलिए उसे सभी अच्छे मार्क्स मिले।
eng_Latn: He is very hardworking and understanding, so he got all the good mark

### Indic to Indic Example


In [ ]:
indic_indic_ckpt_dir = "ai4bharat/indictrans2-indic-indic-1B"  # ai4bharat/indictrans2-indic-indic-dist-320M
indic_indic_tokenizer, indic_indic_model = initialize_model_and_tokenizer(indic_indic_ckpt_dir, "indic-indic", quantization)

ip = IndicProcessor(inference=True)

hi_sents = [
    'जब लाइफ़ हो आउट ऑफ कंट्रोल होठों को करके गोल होठों को करके गोल सीटी बजाके बोल आल इज़ वेल',
    "जब मैं छोटा था, मैं हर रोज़ पार्क जाता था।",
    "उसके पास बहुत सारी पुरानी किताबें हैं, जिन्हें उसने अपने दादा-परदादा से विरासत में पाया।",
    "मुझे समझ में नहीं आ रहा कि मैं अपनी समस्या का समाधान कैसे ढूंढूं।",
    "वह बहुत मेहनती और समझदार है, इसलिए उसे सभी अच्छे मार्क्स मिले।",
    "हमने पिछले सप्ताह एक नई फिल्म देखी जो कि बहुत प्रेरणादायक थी।",
    "अगर तुम मुझे उस समय पास मिलते, तो हम बाहर खाना खाने चलते।",
    "वह अपनी दीदी के साथ बाजार गयी थी ताकि वह नई साड़ी खरीद सके।",
    "राज ने मुझसे कहा कि वह अगले महीने अपनी नानी के घर जा रहा है।",
    "सभी बच्चे पार्टी में मज़ा कर रहे थे और खूब सारी मिठाइयाँ खा रहे थे।",
    "मेरे मित्र ने मुझे उसके जन्मदिन की पार्टी में बुलाया है, और मैं उसे एक तोहफा दूंगा।",
    "उसकी क्या बात कर रहे हो, वह तो मेरे खून का प्यासा हो गया है।",
    "आतंकवादियों को देखकर मेरा तो खून ठण्डा पड़ गया।",
    "आई.पी.एस पास करके दीक्षा ने सचमुच गढ़ फतह कर लिया।",
    "कार्तिक गधे को बाप बनाना अच्छी तरह जानता हैं।",
    "अभी आपको मेरी बात समझ में नहीं आ रही, जब जूते पड़ेंगे तब समझ में आएगी।",
    "जहाँ चाह, वहाँ राह।",
    "हर कड़ी में राह होती है।",
    "काम का बोझ भारी होता है।",
    "रोम एक दिन में नहीं बना था।",
    "अब पछताए होत क्या जब चिड़ीया चुग गई खेत।",
    "हर चमकते चीज सोना नहीं होती।",
    "दो गलत सही नहीं बनाते।",
    "जहाँ धूम है, वहाँ आग होगी।",
    "विद्यार्थी स्कूल जाने का समय है। यह बहुत महत्वपूर्ण है कि आप समय पर स्कूल पहुंचें ताकि आप अपने पढ़ाई में निरंतरता बनाए रख सकें। स्कूल जाना आपके शिक्षा के लिए आवश्यक है और यह आपके भविष्य को सुनिश्चित करने में मदद कर सकता है। इसलिए, आप आज के दिन को सकारात्मकता के साथ शुरू करें और स्कूल के लिए समय पर निकलें। शुभ कामनाएं!",
    "शुभ प्रभात! आपका दिन सुखमय और सफल हो",
    "दिव्यांग व्यक्तियों की मदद करना सामाजिक जिम्मेदारी है।",
    "इन व्यक्तियों को समाज में समाहित करने और उन्हें समर्थ बनाने के लिए हमें सकारात्मक कदम उठाने चाहिए",
    "दिव्यांग व्यक्तियों के साथ सहानुभूति और समर्थन बनाए रखना अत्यंत महत्वपूर्ण है।",
    "उन्हें समाज में पूरी तरह से शामिल करना और उनकी सामाजिक समाहिति को बढ़ावा देना।",
    "उन्हें उच्च शिक्षा और प्रशिक्षण के अवसरों का भी लाभ दिलाना।",
    "सरकार और सामाजिक संगठनों से उनके लिए सुविधाएं और योजनाएं मांगना और उन्हें इनका लाभ उठाने में मदद करना।",
    "उन्हें रोजगार के अवसरों को सुझाना और उन्हें समर्थ बनाने के लिए तकनीकी और व्यावासायिक प्रशिक्षण प्रदान करना।",
    "सामाजिक संगठनों में सक्रियता दिखाना और दिव्यांगता के मुद्दों पर जागरूकता फैलाना।",
    "यह सभी कदम सामूहिक रूप से समृद्धि और समाज में समाहिति की दिशा में सहायक हो सकते हैं।",
    #"कनक कनक ते सौ गुनी, मादकता अधिकाय। वा खाए बौराय जग, या पाए बौराय।",
    "उसकी मुस्कान सूर्य की किरणों की तरह चमक रही थी।",
    "समय एक अजीब सा दरवाज़ा है, जो कभी फिर नहीं खुलता।",
    "बादल बढ़ रहे हैं, बारिश बरस रही है।",
    "बस बुढ़िया बैठी बाल कांपा कर बोली।",
    "बाग़ के फूलों ने रात को महका दिया।",
    "गंगा जी की धारा, स्वर्ग से भी प्यारी है।",
    "सुरमा स्वर्ग से सजीवनी लेकर आई।",
    "उसकी आँखों में आसमान का रंग और उसके बोल में दरिया की गहराई थी",
    "हनुमान की पूंछ में लगन ना पाई आग, लंका सारी जल गई, गए निशाचर भग"
]
src_lang, tgt_lang = "hin_Deva", "pan_Guru"
mr_translations = batch_translate(hi_sents, src_lang, tgt_lang, indic_indic_model, indic_indic_tokenizer, ip)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(hi_sents, mr_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
del indic_indic_tokenizer, indic_indic_model


hin_Deva - pan_Guru
hin_Deva: जब लाइफ़ हो आउट ऑफ कंट्रोल होठों को करके गोल होठों को करके गोल सीटी बजाके बोल आल इज़ वेल
pan_Guru: ਜਦੋਂ ਜ਼ਿੰਦਗੀ ਨਿਯੰਤਰਣ ਤੋਂ ਬਾਹਰ ਹੁੰਦੀ ਹੈ ਤਾਂ ਬੁੱਲ੍ਹ ਗੋਲ ਹੁੰਦੇ ਹਨ, ਬੁੱਲ੍ਹ ਗੋਲ ਹੁੰਦੇ ਹਨ, ਸੀਟੀ ਵਜਾਉਂਦੇ ਹਨ।
hin_Deva: जब मैं छोटा था, मैं हर रोज़ पार्क जाता था।
pan_Guru: ਜਦੋਂ ਮੈਂ ਛੋਟਾ ਸੀ ਤਾਂ ਮੈਂ ਹਰ ਰੋਜ਼ ਪਾਰਕ ਜਾਂਦਾ ਸੀ।
hin_Deva: उसके पास बहुत सारी पुरानी किताबें हैं, जिन्हें उसने अपने दादा-परदादा से विरासत में पाया।
pan_Guru: ਉਸ ਕੋਲ ਬਹੁਤ ਸਾਰੀਆਂ ਪੁਰਾਣੀਆਂ ਕਿਤਾਬਾਂ ਹਨ, ਜੋ ਉਸ ਨੂੰ ਆਪਣੇ ਦਾਦਾ-ਦਾਦੀ ਤੋਂ ਵਿਰਾਸਤ ਵਿੱਚ ਮਿਲੀਆਂ ਹਨ।
hin_Deva: मुझे समझ में नहीं आ रहा कि मैं अपनी समस्या का समाधान कैसे ढूंढूं।
pan_Guru: ਮੈਨੂੰ ਸਮਝ ਨਹੀਂ ਆਉਂਦੀ ਕਿ ਮੈਂ ਆਪਣੀ ਸਮੱਸਿਆ ਦਾ ਹੱਲ ਕਿਵੇਂ ਲੱਭ ਸਕਦਾ ਹਾਂ।
hin_Deva: वह बहुत मेहनती और समझदार है, इसलिए उसे सभी अच्छे मार्क्स मिले।
pan_Guru: ਉਹ ਬਹੁਤ ਮਿਹਨਤੀ ਅਤੇ ਸਮਝਦਾਰ ਹੈ, ਇਸ ਲਈ ਉਸ ਨੂੰ ਸਾਰੇ ਚੰਗੇ ਅੰਕ ਮਿਲੇ।
hin_Deva: हमने पिछले सप्ताह एक नई फिल्म देखी जो कि बहुत प्रेरणादायक थी।
pan_Guru: ਅਸੀਂ ਪਿਛਲੇ ਹਫ਼ਤੇ ਇੱਕ ਨਵੀਂ ਫਿਲਮ ਵੇਖੀ ਜੋ ਬਹੁਤ ਪ੍ਰੇਰਣਾਦਾਇਕ ਸੀ।
hin_Deva: अगर त

In [ ]:
from google.colab import drive

In [ ]:
%cd content/drive/MyDrive/IndicTrans2/

[Errno 2] No such file or directory: 'content/drive/MyDrive/IndicTrans2/'
/content


In [4]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print("Current working directory:", current_directory)

Current working directory: /content


In [5]:
import os

# Get the current working directory
current_directory = os.getcwd()

# List all the contents of the current directory
directory_contents = os.listdir(current_directory)

# Print the contents of the directory
print("Contents of the current directory:")
for item in directory_contents:
    print(item)

Contents of the current directory:
.config
sample_data


In [6]:
import os

# Specify the name of the folder you want to navigate to
folder_name = "sample_data"

# Get the current working directory
current_directory = os.getcwd()

# Construct the path to the folder you want to navigate to
folder_path = os.path.join(current_directory, folder_name)

# Change the current working directory to the specified folder
os.chdir(folder_path)

# Confirm that you are now in the specified folder
print("Current working directory:", os.getcwd())


Current working directory: /content/sample_data


In [7]:
import os

# Get the current working directory
current_directory = os.getcwd()

# List all the contents of the current directory
directory_contents = os.listdir(current_directory)

# Print the contents of the directory
print("Contents of the current directory:")
for item in directory_contents:
    print(item)

Contents of the current directory:
anscombe.json
README.md
california_housing_train.csv
mnist_test.csv
mnist_train_small.csv
california_housing_test.csv
